Follow the below steps to run the notebook in your local environment
*   https://research.google.com/colaboratory/local-runtimes.html


The original notebook is at -> https://colab.research.google.com/drive/1eFjSlaSe6mqM32JAMm3jxEgGRe0Yc6a4?usp=sharing


In [ ]:
!pip install pandas

In [207]:
import requests
import pandas as pd
import io
import json

In [ ]:
# Miscellaneous functions for json flattening

def getFlattenedDf(displayColumns, flattenColumns, convertedDf, jsonList, stop):
  df = convertedDf.reindex(columns = convertedDf.columns.tolist() + displayColumns).fillna(0)
  df = pd.DataFrame(df)
  for colName in flattenColumns:
    for i in range(stop):
      nested = jsonList[i][colName]
      if nested:
        if isinstance(nested, list):
          for n in displayColumns:
            df.loc[i, n] = nested[0][n]
        else:
          for n in displayColumns:
            df.loc[i, n] = nested[n] 
  return df;

def flattenCurrentOrdersJson(res_cart):
  df = pd.DataFrame([res_cart])
  temp_df = getFlattenedDf(["name"], ["store"], df, [res_cart], len([res_cart])).rename(columns={"name": "store_name", "_id":"order_id", "delivery_fee":"order_delivery_fee", "total_amount":"total_order_amount"}).drop(['store'], axis=1)
  temp_df = getFlattenedDf(["name", "street", "city", "country", "postcode"], ["shipping_address"], temp_df, [res_cart], len([res_cart])).rename(columns={"name": "customer_name"}).drop(["shipping_address"], axis=1)
  prod_df = pd.DataFrame(res_cart["items"]).rename(columns={"name": "product_name", "quantity": "product_quantity", "total":"item_total"})
  if 'ETA' in res_cart:
    temp_df = getFlattenedDf(["ETA", "distance"], ["deliveries"], temp_df, [res_cart], len([res_cart])).rename(columns={"distance": "shipping_address_distance"})
  temp_df = temp_df.merge(prod_df, how='cross')
  temp_df = temp_df.drop(['customer_id', 'store_id', 'items', 'created_at', 'total_item_price', 'is_fresh', "order_status_histories", "deliveries"], axis=1, errors='ignore')
  return temp_df;

**Application Endpoints**

In [ ]:
# Lets consider customer 63bebdda1cd050518854522c
# Customer checks the list of products; customer id is passed to get products from the nearest store/warehouse
get_fresh_products_url = "http://localhost:3000/api/products/fresh/63bebdda1cd050518854522c";
get_other_products_url = "http://localhost:3000/api/products/other/63bebdda1cd050518854522c";
get_fresh_product_using_name = "http://localhost:3000/api/products/fresh/63bebdda1cd050518854522c?name=Croissant";
get_other_product_using_name = "http://localhost:3000/api/products/other/63bebdda1cd050518854522c?name=Men Are from Mars, Women Are from Venus";
get_fresh_product_using_category = "http://localhost:3000/api/products/fresh/63bebdda1cd050518854522c?category=Bakery";
get_other_product_using_category = "http://localhost:3000/api/products/other/63bebdda1cd050518854522c?category=Book";

# add customer details
customer_url = "http://localhost:3000/api/customers";
create_address = "http://localhost:3000/api/customer_addresses"

# Order placement cycle for the customer
get_recommendation_url = "http://localhost:3000/api/product-recommendations";
post_addCart_url = "http://localhost:3000/api/order/current_orders";
customer_cart = "http://localhost:3000/api/order/current_orders/cart/"; # Customer should be able to add/remove products to an existing order in the cart
put_checkout_url = "http://localhost:3000/api/order/current_orders/checkout/";
get_orderDetails_url ="http://localhost:3000/api/order/current_orders/current-order-list/63bebdda1cd050518854522c?size=25&page=1";
put_confirmOrder_url = "http://localhost:3000/api/order/current_orders/seller/confirm-order/";
put_pickupOrder_url = "http://localhost:3000/api/order/current_orders/partner/pick-up-order/";
put_trackOrder_url = "http://localhost:3000/api/order/current_orders/track-order/";
put_delivered_url = "http://localhost:3000/api/order/current_orders/partner/order-delivered/";
get_pastOrder_url = "http://localhost:3000/api/order/past_orders/63bebdda1cd050518854522c";
post_partnerRating_url = "http://localhost:3000/api/partners/rating";
put_productRating_url = "http://localhost:3000/api/order/current_orders/product-rating/"

# Manager/Admin related endpoints
partner_url = "http://localhost:3000/api/partners/"
stores_url = "http://localhost:3000/api/stores"
new_product = "http://localhost:3000/api/products/"
product_desc = "http://localhost:3000/api/product/description"

**Customer creating an account**

In [ ]:
# A customer can create an account using basic details
create_customer_data = {
  "first_name": "Customer",
  "last_name": "New",
  "gender": "Female",
  "age": 23,
  "phone": "790-134-8888",
  "email": "newcustomer1997@gmail.com"
}
res_create = requests.post(customer_url, json=create_customer_data).json()
customerID = res_create["_id"]
display(pd.DataFrame([res_create]))

,_id,first_name,last_name,gender,age,phone,email
0,63c15011d91a520465cd37e2,Customer,New,Female,23,790-134-8888,newcustomer1997@gmail.com


**Customer adding an address**

In [ ]:
# Customers can add addresses
create_customer_addr_data = {
  "customer_id": customerID,
  "type": "Shipping",
  "unit_no": "19",
  "street": "Worthing Street ",
  "city": "Manchester",
  "country": "United Kingdom",
  "postcode": "M14 7PR",
  "location": {
    "type": "Point",
    "coordinates": [
      53.450008,
      -2.23165
    ]
  }
}
res_create_addre = requests.post(create_address, json=create_customer_addr_data).json()
display(pd.DataFrame([res_create_addre]))

,_id,customer_id,type,unit_no,street,city,country,postcode,location
0,63c15011d91a520465cd37e3,63c15011d91a520465cd37e2,Shipping,19,Worthing Street,Manchester,United Kingdom,M14 7PR,"{'type': 'Point', 'coordinates': [53.450008, -..."


**Manager getting all customer details**

In [ ]:
# A manager can get the details of all customer for analytics purposes
res_cust = requests.get(customer_url).json()
display(pd.DataFrame(res_cust))

,_id,age,email,first_name,gender,last_name,phone
0,63bebd9d1cd0505188545225,29,namjoon@gmail.com,Kim,Male,Namjoon,677-807-304
1,63bebdaa1cd0505188545226,31,seokjin@gmail.com,Kim,Male,Seokjin,336-320-440
2,63bebdb11cd0505188545227,30,yoongi@gmail.com,Min,Male,Yoongi,197-150-840
3,63bebdbf1cd0505188545228,29,hoseok@gmail.com,Jung,Male,Hoseok,375-524-502
4,63bebdc61cd0505188545229,28,jimin@gmail.com,Park,Male,Jimin,657-932-370
5,63bebdcc1cd050518854522a,28,taehyung@gmail.com,Kim,Male,Taehyung,315-714-305
6,63bebdd31cd050518854522b,26,jungkook@gmail.com,Jeon,Male,Jungkook,156-546-620
7,63bebdda1cd050518854522c,28,jisoo@gmail.com,Kim,Female,Jisoo,458-674-798
8,63bebde21cd050518854522d,27,jennie@gmail.com,Kim,Female,Jennie,981-290-616
9,63bebde91cd050518854522e,26,rose@gmail.com,Park,Female,Rose,137-539-639


**1. First use case of a customer creating an order**

**Customer getting reccommended products**

In [ ]:
request_body = {
    "customer_id": "63bebdda1cd050518854522c"
}

res_rec = requests.get(get_recommendation_url, json=request_body).json()
reccomendations = res_rec["recommendations"]
display(pd.DataFrame(reccomendations))

,category,name,selling_price,store_id,stock,rating,product_id,store_name
0,Home Appliances,Rice Cooker,29.00,63be13c2218d0176787ca58f,40,5.00,63bedeb4905ece42d0d08656,Amazon Warehouse 2
1,CD,Speak Now,7.50,63be13c2218d0176787ca58f,28,4.67,63bedcf1905ece42d0d08652,Amazon Warehouse 2
2,CD,Lover,11.69,63be13c2218d0176787ca58f,69,4.38,63bedae23121db4da02d5009,Amazon Warehouse 2
3,Home Appliances,Microwave,55.00,63be13c2218d0176787ca58f,88,4.33,63bede9b905ece42d0d08653,Amazon Warehouse 2
4,CD,Taylor Swift,8.99,63be13c2218d0176787ca58f,21,4.17,63bedadb3121db4da02d5008,Amazon Warehouse 2
5,Mobile Phones,IPhone 14 Pro Max 128Gb,1199.00,63be13c2218d0176787ca58f,208,4.17,63bedc5e905ece42d0d0864a,Amazon Warehouse 2
6,Books,Harry Potter: The Complete Collection,35.00,63be13c2218d0176787ca58f,181,4.14,63bed9443121db4da02d4fff,Amazon Warehouse 2
7,Mobile Phones,IPhone 14 Pro 128Gb,1099.00,63be13c2218d0176787ca58f,146,4.14,63bedc54905ece42d0d08649,Amazon Warehouse 2
8,Mobile Phones,Samsung Galaxy Z Fold4 Ultra 256Gb,1649.00,63be13c2218d0176787ca58f,149,4.14,63bedc6e905ece42d0d0864c,Amazon Warehouse 2
9,Mobile Phones,IPhone 14 Pro 256Gb,1209.00,63be13c2218d0176787ca58f,312,4.13,63bedc4c905ece42d0d08648,Amazon Warehouse 2


**Customer searching for fresh and other products**


In [ ]:
final_column_names = ["_id", "name", "category", "selling_price", "average_rating", "number_of_ratings", "stock", "store_id"]

# Gresh fresh products from the nearest Morrison's store
res_fresh = requests.get(get_fresh_products_url).json()
flattened_df_1 = getFlattenedDf(["stock", "store_id"], ["stocks"], pd.DataFrame(res_fresh), res_fresh, len(res_fresh))
display(flattened_df_1[final_column_names])

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# Gresh other products from the nearest Amazone warehouse
res_other = requests.get(get_other_products_url).json()
flattened_df_2 = getFlattenedDf(["stock", "store_id"], ["stocks"], pd.DataFrame(res_other), res_other, len(res_other))
display(flattened_df_2[final_column_names])

,_id,name,category,selling_price,average_rating,number_of_ratings,stock,store_id
0,63bee1a1905ece42d0d0865d,Butter Croissant,Bakery,0.95,3.67,15.0,830.0,63be1307218d0176787ca583
1,63bee21b905ece42d0d0865f,Pain au Chocolat,Bakery,1.00,3.00,10.0,100.0,63be1307218d0176787ca583
2,63bee227905ece42d0d08660,Chocolate Muffin,Bakery,1.50,2.92,12.0,14.0,63be1307218d0176787ca583
3,63bee23b905ece42d0d08661,Sourdough,Bakery,2.00,3.63,16.0,10.0,63be1307218d0176787ca583
4,63bee417905ece42d0d08662,Coca Cola,Drinks,1.50,4.17,12.0,200.0,63be1307218d0176787ca583
5,63bee424905ece42d0d08663,Pepsi,Drinks,1.50,4.08,12.0,12.0,63be1307218d0176787ca583
6,63bee42d905ece42d0d08664,Fanta,Drinks,1.23,4.09,11.0,6.0,63be1307218d0176787ca583
7,63bee435905ece42d0d08665,Orange Juice,Drinks,1.30,3.92,12.0,17.0,63be1307218d0176787ca583
8,63bee43f905ece42d0d08666,Mango Juice,Drinks,1.20,3.70,10.0,29.0,63be1307218d0176787ca583
9,63bee628905ece42d0d08667,Chillies,Fruits and Vegetables,1.50,3.83,6.0,27.0,63be1307218d0176787ca583


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,_id,name,category,selling_price,average_rating,number_of_ratings,stock,store_id
0,63bed8c13121db4da02d4fec,"Men Are from Mars, Women Are from Venus",Books,3.99,3.00,5,296.0,63be13c2218d0176787ca58f
1,63bed9043121db4da02d4ff8,Doctor Sleep,Books,4.99,3.70,10,14.0,63be13c2218d0176787ca58f
2,63bed90b3121db4da02d4ff9,The 7 Habits of Highly Effective People,Books,12.00,3.70,10,120.0,63be13c2218d0176787ca58f
3,63bed9173121db4da02d4ffa,Freakonomics,Books,8.85,3.00,5,70.0,63be13c2218d0176787ca58f
4,63bed9203121db4da02d4ffb,USBorne Economics for Beginners,Books,9.19,2.60,5,68.0,63be13c2218d0176787ca58f
5,63bed92a3121db4da02d4ffc,The Tiger Who Came to Tea,Books,4.00,3.75,8,88.0,63be13c2218d0176787ca58f
6,63bed93c3121db4da02d4ffe,Atomic Habits,Books,9.00,3.80,10,118.0,63be13c2218d0176787ca58f
7,63bed9443121db4da02d4fff,Harry Potter: The Complete Collection,Books,35.00,4.14,7,181.0,63be13c2218d0176787ca58f
8,63bed9573121db4da02d5001,Jeff Bezos: Biography of a Billionaire Busines...,Books,9.44,3.20,5,290.0,63be13c2218d0176787ca58f
9,63bedaa03121db4da02d5002,Midnights,CD,10.00,4.00,9,23.0,63be13c2218d0176787ca58f


**Customer searching for fresh and other products using name**

In [ ]:
# Search for fresh products using the name
res_fresh_name = requests.get(get_fresh_product_using_name).json()
flattened_df_1 = getFlattenedDf(["stock", "store_id"], ["stocks"], pd.DataFrame(res_fresh_name), res_fresh_name, len(res_fresh_name))
display(flattened_df_1[final_column_names])


print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")


# Search for other products using the name
res_other_name = requests.get(get_other_product_using_name).json()
flattened_df_2 = getFlattenedDf(["stock", "store_id"], ["stocks"], pd.DataFrame(res_other_name), res_other_name, len(res_other_name))
display(flattened_df_2[final_column_names])

,_id,name,category,selling_price,average_rating,number_of_ratings,stock,store_id
0,63bee1a1905ece42d0d0865d,Butter Croissant,Bakery,0.95,3.67,15,830.0,63be1307218d0176787ca583


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,_id,name,category,selling_price,average_rating,number_of_ratings,stock,store_id
0,63bed8c13121db4da02d4fec,"Men Are from Mars, Women Are from Venus",Books,3.99,3,5,296.0,63be13c2218d0176787ca58f


**Customer searching for fresh and other products using category**

In [ ]:
# Search for fresh products using the category
res_fresh_cat = requests.get(get_fresh_product_using_category).json()
flattened_df_1 = getFlattenedDf(["stock", "store_id"], ["stocks"], pd.DataFrame(res_fresh_cat), res_fresh_cat, len(res_fresh_cat))
display(flattened_df_1[final_column_names])


print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")


# Search for other products using the category
res_other_cat = requests.get(get_other_product_using_category).json()
flattened_df_2 = getFlattenedDf(["stock", "store_id"], ["stocks"], pd.DataFrame(res_other_cat), res_other_cat, len(res_other_cat))
display(flattened_df_2[final_column_names])

,_id,name,category,selling_price,average_rating,number_of_ratings,stock,store_id
0,63bee1a1905ece42d0d0865d,Butter Croissant,Bakery,0.95,3.67,15,830.0,63be1307218d0176787ca583
1,63bee21b905ece42d0d0865f,Pain au Chocolat,Bakery,1.00,3.00,10,100.0,63be1307218d0176787ca583
2,63bee227905ece42d0d08660,Chocolate Muffin,Bakery,1.50,2.92,12,14.0,63be1307218d0176787ca583
3,63bee23b905ece42d0d08661,Sourdough,Bakery,2.00,3.63,16,10.0,63be1307218d0176787ca583


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,_id,name,category,selling_price,average_rating,number_of_ratings,stock,store_id
0,63bed8c13121db4da02d4fec,"Men Are from Mars, Women Are from Venus",Books,3.99,3.00,5,296.0,63be13c2218d0176787ca58f
1,63bed9043121db4da02d4ff8,Doctor Sleep,Books,4.99,3.70,10,14.0,63be13c2218d0176787ca58f
2,63bed90b3121db4da02d4ff9,The 7 Habits of Highly Effective People,Books,12.00,3.70,10,120.0,63be13c2218d0176787ca58f
3,63bed9173121db4da02d4ffa,Freakonomics,Books,8.85,3.00,5,70.0,63be13c2218d0176787ca58f
4,63bed9203121db4da02d4ffb,USBorne Economics for Beginners,Books,9.19,2.60,5,68.0,63be13c2218d0176787ca58f
5,63bed92a3121db4da02d4ffc,The Tiger Who Came to Tea,Books,4.00,3.75,8,88.0,63be13c2218d0176787ca58f
6,63bed93c3121db4da02d4ffe,Atomic Habits,Books,9.00,3.80,10,118.0,63be13c2218d0176787ca58f
7,63bed9443121db4da02d4fff,Harry Potter: The Complete Collection,Books,35.00,4.14,7,181.0,63be13c2218d0176787ca58f
8,63bed9573121db4da02d5001,Jeff Bezos: Biography of a Billionaire Busines...,Books,9.44,3.20,5,290.0,63be13c2218d0176787ca58f


**Customer creating an order**

In [ ]:
# This customer adds 1 Butter croissant and 2 coca cola to her cart from the nearest store; 
# Following is her cart view 
request_body = {
    "customer_id": "63bebdda1cd050518854522c",
    "store_id": "63be1307218d0176787ca583",
    "items": [
        {
            "product_id": "63bee1a1905ece42d0d0865d",
            "quantity": 1
        },
        {
            "product_id": "63bee417905ece42d0d08662",
            "quantity": 2            
        }
    ]
}

res_cart = requests.post(post_addCart_url, json=request_body).json()
orderID = res_cart["_id"]
display(flattenCurrentOrdersJson(res_cart))
# order_status is set to "on-cart", 
# customer has yet to make payments on the order (is_paid=False)

,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-cart,False,63c15013d91a520465cd37e5,5.34,9.29,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,1,Butter Croissant,0.95,0.95
1,on-cart,False,63c15013d91a520465cd37e5,5.34,9.29,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee417905ece42d0d08662,2,Coca Cola,1.50,3.00


**Customer getting all their orders**

In [ ]:
# This request can be paginated to adjust the size and page number
res_orders = requests.get(get_orderDetails_url).json()
display(pd.DataFrame(res_orders["current_order"]))

,order_status,is_paid,_id,customer_id,store_id,store,items,shipping_address,delivery_fee,created_at,total_item_price,is_fresh,total_amount,order_status_histories,deliveries,partners
0,on-cart,False,63c04f2f90aea1449644ec29,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T18:19:27.277Z,3.95,True,9.29,NaN,NaN,NaN
1,payment-confirmed,True,63c04f4190aea1449644ec35,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T18:19:45.456Z,3.95,True,9.29,"[{'name': 'payment-confirmed', 'created_at': '...",NaN,NaN
2,on-cart,False,63c05a7530a1de5331ed7ac7,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T19:07:33.654Z,3.95,True,9.29,NaN,NaN,NaN
3,on-cart,False,63c05b7730a1de5331ed7ae7,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T19:11:51.273Z,3.95,True,9.29,NaN,NaN,NaN
4,on-delivery,True,63c07d7330a1de5331ed7c82,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T21:36:51.408Z,3.95,True,9.29,"[{'name': 'payment-confirmed', 'created_at': '...",{'order_delivery_status_histories': [{'name': ...,63c08bff292b474520345978
5,on-delivery,True,63c08358846e4d43b4e06a1b,63bebdda1cd050518854522c,63be13c2218d0176787ca58f,"{'name': 'Amazon Warehouse 2', 'location': {'t...","[{'product_id': '63bedea3905ece42d0d08654', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",3.34,2023-01-12T22:02:00.944Z,13.00,False,16.34,"[{'name': 'payment-confirmed', 'created_at': '...",{'order_delivery_status_histories': [{'name': ...,NaN
6,on-cart,False,63c08e0ff0ab5e53abc7c796,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T22:47:43.742Z,3.95,True,9.29,NaN,NaN,NaN
7,on-cart,False,63c092fef0ab5e53abc7c798,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T23:08:46.558Z,3.95,True,9.29,NaN,NaN,NaN
8,on-cart,False,63c095ddf0ab5e53abc7c79a,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T23:21:01.054Z,3.95,True,9.29,NaN,NaN,NaN
9,on-cart,False,63c095e3f0ab5e53abc7c79c,63bebdda1cd050518854522c,63be1307218d0176787ca583,"{'name': 'Morrison 5', 'location': {'type': 'P...","[{'product_id': '63bee1a1905ece42d0d0865d', 'q...","{'name': 'Kim Jisoo', 'unit_no': '46', 'street...",5.34,2023-01-12T23:21:07.671Z,3.95,True,9.29,NaN,NaN,NaN


**Customer adding new and existing products to the cart**

In [ ]:
# Adding an existing product to the cart; quantity & price should be incremented
request_body = {
    "product_id": "63bee1a1905ece42d0d0865d",
    "quantity": 2
}
res_after_1 = requests.put(customer_cart + orderID, json=request_body).json()

# Adding a new product to the cart; number of items in the cart will increase
request_body = {
    "product_id": "63bee21b905ece42d0d0865f",
    "quantity": 1
}
res_after_2 = requests.put(customer_cart + orderID, json=request_body).json()

In [ ]:
print("Adding an existing product")
display(flattenCurrentOrdersJson(res_after_1))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

print("Adding a new product")
display(flattenCurrentOrdersJson(res_after_2))

Adding an existing product


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-cart,False,63c15013d91a520465cd37e5,5.34,11.19,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,3,Butter Croissant,0.95,2.85
1,on-cart,False,63c15013d91a520465cd37e5,5.34,11.19,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee417905ece42d0d08662,2,Coca Cola,1.50,3.00


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
Adding a new product


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-cart,False,63c15013d91a520465cd37e5,5.34,12.19,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,3,Butter Croissant,0.95,2.85
1,on-cart,False,63c15013d91a520465cd37e5,5.34,12.19,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee417905ece42d0d08662,2,Coca Cola,1.50,3.00
2,on-cart,False,63c15013d91a520465cd37e5,5.34,12.19,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee21b905ece42d0d0865f,1,Pain au Chocolat,1.00,1.00


**Customer deleting product from the cart**

In [ ]:
# Deleting one product from the order; should reduce the total amount and total item price (i.e., price excluding the delivery)
# Deleting the only item should delete the order because there are no items left
productID = "63bee417905ece42d0d08662" # Deleting Cocal Cola
res_after_del = requests.delete(customer_cart + orderID + "/" + productID).json()
print(res_after_del["msg"])

Product/Order successfully deleted


**Customer ordering a product, adding it to the cart and making payment**

In [ ]:
# Customer makes the payment 
res_cart = requests.put(put_checkout_url + orderID).json()
print(res_cart)

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# order confirmation is done
res_confirm = requests.put(put_confirmOrder_url + orderID).json()
display(flattenCurrentOrdersJson(res_confirm))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# order will be assigned to the nearest partner, as can be seen in the partners column 
res_pickup = requests.put(put_pickupOrder_url + orderID).json()
partnerID = res_pickup["partners"]
display(flattenCurrentOrdersJson(res_pickup))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# the order is sent to the nearest available driver, the order status is changed to on-delivery, and the customer can view ETA of her items
response = requests.put(put_trackOrder_url + orderID).json()
display(flattenCurrentOrdersJson(response))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# product is delivered, status is changed to delivered, and data is moved to past orders
res_delivered = requests.put(put_delivered_url + orderID).json()
print("Acknowledgement receipt:" + str(res_delivered))

{'message': 'Payment confirmed'}
----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,partners,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,order-confirmed,True,63c15013d91a520465cd37e5,5.34,9.19,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,3,Butter Croissant,0.95,2.85
1,order-confirmed,True,63c15013d91a520465cd37e5,5.34,9.19,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee21b905ece42d0d0865f,1,Pain au Chocolat,1.00,1.00


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,partners,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-delivery,True,63c15013d91a520465cd37e5,5.34,9.19,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,3,Butter Croissant,0.95,2.85
1,on-delivery,True,63c15013d91a520465cd37e5,5.34,9.19,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee21b905ece42d0d0865f,1,Pain au Chocolat,1.00,1.00


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,partners,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-delivery,True,63c15013d91a520465cd37e5,5.34,9.19,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,3,Butter Croissant,0.95,2.85
1,on-delivery,True,63c15013d91a520465cd37e5,5.34,9.19,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee21b905ece42d0d0865f,1,Pain au Chocolat,1.00,1.00


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
Acknowledgement receipt:{'acknowledge': True}


**Customer checking past orders**

In [ ]:
res_pastOrders = requests.get(get_pastOrder_url).json()
display(pd.DataFrame(res_pastOrders))

,_id,end_date,orders,start_date
0,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf73c8013b1a5cd0759fc4', 'customer_...",2023-01-12T00:00:00.000Z
1,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf7cc8f0a2c52eb0ea3493', 'customer_...",2023-01-12T00:00:00.000Z
2,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf87b9c216d852749df5b4', 'customer_...",2023-01-12T00:00:00.000Z
3,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf949216e9455960045b1b', 'customer_...",2023-01-12T00:00:00.000Z
4,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf9d1fde3a865eec48fb8d', 'customer_...",2023-01-12T00:00:00.000Z
5,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63c03dba5c4b0b4858761606', 'customer_...",2023-01-12T00:00:00.000Z
6,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63c05c5630a1de5331ed7aff', 'customer_...",2023-01-12T00:00:00.000Z
7,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63c09385292b4745203459d1', 'customer_...",2023-01-12T00:00:00.000Z
8,63c0b28aec196abcb5fa913b,2023-01-13T23:59:59.999Z,"{'_id': '63c0af0871f5515e44c598bf', 'customer_...",2023-01-13T00:00:00.000Z
9,63c0b28aec196abcb5fa913b,2023-01-13T23:59:59.999Z,"{'_id': '63c0b9e3d5182e606b3af88d', 'customer_...",2023-01-13T00:00:00.000Z


**Customer rating the product and the partner**

In [ ]:
# after the item is delivered, the customer is prompted to rate delivery partner
request_body = {
    "partner_id": partnerID,
    "order_id":orderID,
    "rating": 2
}
res_partnerRating = requests.post(post_partnerRating_url, json=request_body).text
display(pd.read_json(res_partnerRating, lines=True))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# the customer is also prompted to rate the products
request_body = [{
            "product_id": "63bee227905ece42d0d08660",
            "rating": 3
        },
        {
            "product_id": "63bee23b905ece42d0d08661",
            "rating": 4          
        }]
res_productRating = requests.put(put_productRating_url + orderID, json=request_body).text
display(pd.read_json(res_productRating, lines=True))

,_id,partner_id,order_id,rating,date
0,63c15014d91a520465cd37ea,63c08da5292b474520345983,63c15013d91a520465cd37e5,2,2023-01-13 12:35:32.788000+00:00


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,message
0,ok


**2. Second use case of a customer creating an order**

In [ ]:
# Order placement cycle for the customer
get_recommendation_url = "http://localhost:3000/api/product-recommendations";
post_addCart_url = "http://localhost:3000/api/order/current_orders";
customer_cart = "http://localhost:3000/api/order/current_orders/cart/"; # Customer should be able to add/remove products to an existing order in the cart
put_checkout_url = "http://localhost:3000/api/order/current_orders/checkout/";
get_orderDetails_url ="http://localhost:3000/api/order/current_orders/";
put_confirmOrder_url = "http://localhost:3000/api/order/current_orders/seller/confirm-order/";
put_pickupOrder_url = "http://localhost:3000/api/order/current_orders/partner/pick-up-order/";
put_trackOrder_url = "http://localhost:3000/api/order/current_orders/track-order/";
put_delivered_url = "http://localhost:3000/api/order/current_orders/partner/order-delivered/";
get_pastOrder_url = "http://localhost:3000/api/order/past_orders/63bebdda1cd050518854522c";
post_partnerRating_url = "http://localhost:3000/api/partners/rating";
put_productRating_url = "http://localhost:3000/api/order/current_orders/product-rating/"

In [ ]:
# This customer adds 1 Spinach and 1 Sourdough bread to her cart from the nearest store; 
# Following is her cart view 
request_body = {
    "customer_id": "63bebdda1cd050518854522c",
    "store_id": "63be1307218d0176787ca583",
    "items": [
        {
            "product_id": "63bee66f905ece42d0d0866b",
            "quantity": 1
        },
        {
            "product_id": "63bee23b905ece42d0d08661",
            "quantity": 1            
        }
    ]
}

res_cart = requests.post(post_addCart_url, json=request_body).json()
orderID = res_cart["_id"]
display(flattenCurrentOrdersJson(res_cart))
# order_status is set to "on-cart", 
# customer has yet to make payments on the order (is_paid=False)

,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-cart,False,63c15015d91a520465cd37ec,5.34,8.09,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee66f905ece42d0d0866b,1,Sourdough,2.00,2.00
1,on-cart,False,63c15015d91a520465cd37ec,5.34,8.09,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee23b905ece42d0d08661,1,Spinach,0.75,0.75


In [ ]:
# Adding an existing product to the cart; quantity & price should be incremented
request_body = {
    "product_id": "63bee66f905ece42d0d0866b",
    "quantity": 1
}
res_after_1 = requests.put(customer_cart + orderID, json=request_body).json()

# Adding a new product to the cart; number of items in the cart will increase
request_body = {
    "product_id": "63bee1a1905ece42d0d0865d",
    "quantity": 2
}
res_after_2 = requests.put(customer_cart + orderID, json=request_body).json()

print("Adding an existing product")
display(flattenCurrentOrdersJson(res_after_1))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

print("Adding a new product")
display(flattenCurrentOrdersJson(res_after_2))

Adding an existing product


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-cart,False,63c15015d91a520465cd37ec,5.34,8.84,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee66f905ece42d0d0866b,2,Sourdough,2.00,2.75
1,on-cart,False,63c15015d91a520465cd37ec,5.34,8.84,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee23b905ece42d0d08661,1,Spinach,0.75,0.75


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
Adding a new product


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-cart,False,63c15015d91a520465cd37ec,5.34,10.74,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee66f905ece42d0d0866b,2,Sourdough,2.00,2.75
1,on-cart,False,63c15015d91a520465cd37ec,5.34,10.74,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee23b905ece42d0d08661,1,Spinach,0.75,0.75
2,on-cart,False,63c15015d91a520465cd37ec,5.34,10.74,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,2,Butter Croissant,0.95,1.90


In [ ]:
# Customer makes the payment 
res_cart = requests.put(put_checkout_url + orderID).json()
print(res_cart)

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# order confirmation is done
res_confirm = requests.put(put_confirmOrder_url + orderID).json()
display(flattenCurrentOrdersJson(res_confirm))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# order will be assigned to the nearest partner, as can be seen in the partners column 
res_pickup = requests.put(put_pickupOrder_url + orderID).json()
partnerID = res_pickup["partners"]
display(flattenCurrentOrdersJson(res_pickup))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# the order is sent to the nearest available driver, the order status is changed to on-delivery, and the customer can view ETA of her items
response = requests.put(put_trackOrder_url + orderID).json()
display(flattenCurrentOrdersJson(response))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# product is delivered, status is changed to delivered, and data is moved to past orders
res_delivered = requests.put(put_delivered_url + orderID).json()
print("Acknowledgement receipt:" + str(res_delivered))

{'message': 'Payment confirmed'}
----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,partners,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,order-confirmed,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee66f905ece42d0d0866b,2,Sourdough,2.00,2.75
1,order-confirmed,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee23b905ece42d0d08661,1,Spinach,0.75,0.75
2,order-confirmed,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,2,Butter Croissant,0.95,1.90


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,partners,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-delivery,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee66f905ece42d0d0866b,2,Sourdough,2.00,2.75
1,on-delivery,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee23b905ece42d0d08661,1,Spinach,0.75,0.75
2,on-delivery,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,2,Butter Croissant,0.95,1.90


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,order_status,is_paid,order_id,order_delivery_fee,total_order_amount,partners,store_name,customer_name,street,city,country,postcode,product_id,product_quantity,product_name,unit_price,item_total
0,on-delivery,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee66f905ece42d0d0866b,2,Sourdough,2.00,2.75
1,on-delivery,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee23b905ece42d0d08661,1,Spinach,0.75,0.75
2,on-delivery,True,63c15015d91a520465cd37ec,5.34,10.74,63c08da5292b474520345983,Morrison 5,Kim Jisoo,Arnside Street,Arnside Street,United Kingdom,M14 7PF,63bee1a1905ece42d0d0865d,2,Butter Croissant,0.95,1.90


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
Acknowledgement receipt:{'acknowledge': True}


In [ ]:
# to check the past orders for this customer
res_pastOrders = requests.get(get_pastOrder_url).json()
display(pd.DataFrame(res_pastOrders))

,_id,end_date,orders,start_date
0,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf73c8013b1a5cd0759fc4', 'customer_...",2023-01-12T00:00:00.000Z
1,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf7cc8f0a2c52eb0ea3493', 'customer_...",2023-01-12T00:00:00.000Z
2,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf87b9c216d852749df5b4', 'customer_...",2023-01-12T00:00:00.000Z
3,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf949216e9455960045b1b', 'customer_...",2023-01-12T00:00:00.000Z
4,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63bf9d1fde3a865eec48fb8d', 'customer_...",2023-01-12T00:00:00.000Z
5,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63c03dba5c4b0b4858761606', 'customer_...",2023-01-12T00:00:00.000Z
6,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63c05c5630a1de5331ed7aff', 'customer_...",2023-01-12T00:00:00.000Z
7,63bf5dd1c0d013af46cacb37,2023-01-12T23:59:59.999Z,"{'_id': '63c09385292b4745203459d1', 'customer_...",2023-01-12T00:00:00.000Z
8,63c0b28aec196abcb5fa913b,2023-01-13T23:59:59.999Z,"{'_id': '63c0af0871f5515e44c598bf', 'customer_...",2023-01-13T00:00:00.000Z
9,63c0b28aec196abcb5fa913b,2023-01-13T23:59:59.999Z,"{'_id': '63c0b9e3d5182e606b3af88d', 'customer_...",2023-01-13T00:00:00.000Z


In [ ]:
# after the item is delivered, the customer is prompted to rate delivery partner
request_body = {
    "partner_id": partnerID,
    "order_id":orderID,
    "rating": 2
}
res_partnerRating = requests.post(post_partnerRating_url, json=request_body).text
display(pd.read_json(res_partnerRating, lines=True))

print("----------------------------------------------------------------------------------------------")
print("----------------------------------------------------------------------------------------------")

# the customer is also prompted to rate the products
request_body = [{
            "product_id": "63bee66f905ece42d0d0866b",
            "rating": 3
        },
        {
            "product_id": "63bee23b905ece42d0d08661",
            "rating": 4          
        },
        {
           "product_id": "63bee1a1905ece42d0d0865d",
            "rating": 4  
        }]
res_productRating = requests.put(put_productRating_url + orderID, json=request_body).text
display(pd.read_json(res_productRating, lines=True))

,_id,partner_id,order_id,rating,date
0,63c15016d91a520465cd37f2,63c08da5292b474520345983,63c15015d91a520465cd37ec,2,2023-01-13 12:35:34.409000+00:00


----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------


,message
0,ok


**Manager/Admin related end-points**

**Manager adding a partner**

In [ ]:
# A customer can create an account using basic details
create_partner = {
  "name": "New Partner",
  "age": 25,
  "gender": "Male",
  "phone": "07341223105",
  "account_number": "24012220",
  "sortcode": "01-11-11",
  "location": {
    "coordinates": [
      53.442782687834125, -2.2906427264145566
    ]
  },
  "is_idle": 1
}
res_partner = requests.post(partner_url, json=create_partner).json()
partnerID = res_partner['_id']
display(pd.DataFrame([res_partner]))

,is_idle,_id,name,age,gender,phone,account_number,sortcode,location
0,True,63c15016d91a520465cd37f3,New Partner,25,Male,07341223105,24012220,01-11-11,"{'type': 'Point', 'coordinates': [53.442782687..."


**Manager updating partner details**

In [ ]:
update_partner = {
    "name": "Khoirunnisa",
    "phone": "777-123-4567",
    "location": {
      "type": "Point",
      "coordinates": [42.356479, -71.068316]
    },
    "is_idle": 1,
    "account_number": "123456789",
    "sortcode": "11-11-11",
    "gender": "Female"
  }

res_partner_upd = requests.put(partner_url + partnerID, json=update_partner).json()
display(pd.DataFrame([res_partner_upd]))

,is_idle,_id,name,age,gender,phone,account_number,sortcode,location
0,True,63c15016d91a520465cd37f3,Khoirunnisa,25,Female,777-123-4567,123456789,11-11-11,"{'type': 'Point', 'coordinates': [42.356479, -..."


**Manager deleting partner details**

In [ ]:
res_partner = requests.delete(partner_url + partnerID).json()
display(pd.DataFrame([res_partner]))

,is_idle,_id,name,age,gender,phone,account_number,sortcode,location
0,True,63c15016d91a520465cd37f3,Khoirunnisa,25,Female,777-123-4567,123456789,11-11-11,"{'type': 'Point', 'coordinates': [42.356479, -..."


**Manager adding a store**

In [ ]:
add_store_body = {
  "name": "Amazon Warehouse 3",
  "is_warehouse": 1,
  "address": {
    "unit_no": "36",
    "street": "Yew Tree Road",
    "city": "Manchester",
    "country": "United Kingdom",
    "postcode": "M14 7PW",
    "location": {
      "type": "Point",
      "coordinates": [
        53.42415423496177, -2.2411835103893765
      ]
    }
  }
}
res_store_add = requests.post(stores_url, json=add_store_body).json()
display(pd.DataFrame([res_store_add]))

,_id,name,is_warehouse,address
0,63c15016d91a520465cd37f4,Amazon Warehouse 3,True,"{'_id': '63c15016d91a520465cd37f5', 'unit_no':..."


**Manager getting store details**

In [ ]:
res_store = requests.get(stores_url).json()
flattened_df_2 = getFlattenedDf(["unit_no", "street", "city", "country", "postcode"], ["address"], pd.DataFrame(res_store), res_store, len(res_store)).drop(["address"], axis=1)
display(pd.DataFrame(flattened_df_2))

,_id,name,is_warehouse,unit_no,street,city,country,postcode
0,63be127a218d0176787ca57b,Morrison 1,False,58,Graham Street,Manchester,United Kingdom,M14 4HG
1,63be12a7218d0176787ca57d,Morrison 2,False,4,Oxford Street,Manchester,United Kingdom,M13 9PL
2,63be12c6218d0176787ca57f,Morrison 3,False,29,Princess Road,Manchester,United Kingdom,M15 6HJ
3,63be12e8218d0176787ca581,Morrison 4,False,47,Lloyd Street North,Manchester,United Kingdom,M15 6SE
4,63be1307218d0176787ca583,Morrison 5,False,56,Wilmslow Road,Manchester,United Kingdom,M14 5LR
5,63be1335218d0176787ca585,Morrison 6,False,20,Lower Park Road,Manchester,United Kingdom,M14 5QY
6,63be1357218d0176787ca587,Morrison 7,False,78,Dickenson Road,Manchester,United Kingdom,M14 5AZ
7,63be137b218d0176787ca589,Morrison 8,False,37,Langley Road,Manchester,United Kingdom,M14 6JH
8,63be13ae218d0176787ca58d,Amazon Warehouse 1,True,103,Brunswick Street,Manchester,United Kingdom,M13 9SR
9,63be13c2218d0176787ca58f,Amazon Warehouse 2,True,55,Monton Street,Manchester,United Kingdom,M14 4LT


**Manager adding a new product to the catalog**

In [ ]:
# Adding new product (fresh or others; use flag) to the catalog
new_prod = {
  "category": "Fruits and Vegetables",
  "name": "Watermelon",
  "cost_price": 0.4,
  "selling_price": 0.75,
  "stocks": [
    {
      "store_id": "63be127a218d0176787ca57b",
      "stock": 8
    },
    {
      "store_id": "63be12a7218d0176787ca57d",
      "stock": 15
    },
    {
      "store_id": "63be12c6218d0176787ca57f",
      "stock": 40
    },
    {
      "store_id": "63be12e8218d0176787ca581",
      "stock": 25
    },
    {
      "store_id": "63be1307218d0176787ca583",
      "stock": 34
    },
    {
      "store_id": "63be1335218d0176787ca585",
      "stock": 35
    },
    {
      "store_id": "63be1357218d0176787ca587",
      "stock": 28
    },
    {
      "store_id": "63be137b218d0176787ca589",
      "stock": 29
    }
  ],
  "is_fresh": 1
}
res_new_prod = requests.post(new_product, json=new_prod).json()
prodID = res_new_prod['_id']
display(pd.DataFrame([res_new_prod]).drop(["stocks"], axis=1))

,_id,category,name,cost_price,selling_price,is_fresh
0,63c15016d91a520465cd37f6,Fruits and Vegetables,Watermelon,0.4,0.75,True


**Manager adding description for the newly added product**

In [ ]:
prod_decribe = {
  "_id": prodID, 
  "product_description": "It's a Watermelon!",
  "product_dimension": {
    "height": "14.2cm",
    "width": "20.8cm",
    "depth": "15.8cm"
  },
  "product_weight": 1,
}
res_new_prod_desc = requests.post(product_desc, json=prod_decribe).json()
display(pd.DataFrame([res_new_prod_desc]))

,_id,product_description,product_weight
0,63c15016d91a520465cd37f6,It's a Watermelon!,1
